In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [2]:
water = pd.read_csv('drinking_water_potability.csv')
water.head()
water = water.dropna()
water.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.41744,8.059332,356.886136,363.266516,18.436525,100.341674,4.628771,0
4,9.092223,181.101509,17978.98634,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.68774,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.71654,7.513408,393.663395,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.09176,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0


In [14]:
np.random.seed(2)
# 80% to be training
train = water.sample(round(water.shape[0]*.8))
# drop the train data from the test df
test = water.drop(train.index)

In [15]:
y = train.filter(items=['Potability'])
X = train.drop('Potability', axis=1)

testY = test.filter(items=['Potability'])
testX = test.drop('Potability', axis=1)

In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV


model = RidgeClassifier()

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.407
Config: {'alpha': 0.0}


C:\Users\yshik\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
ridgeModel = RidgeClassifier(alpha = 0)
predictedY = ridgeModel.fit(X, y).predict(testX)

tp = 0
tn = 0
fp = 0
fn = 0
for ii in range(0, len(predictedY)):
    if predictedY[ii] == 0:
        if testY.iloc[ii][0] == 0:
            tn += 1
        else:
            fn +=1
    else:
        if testY.iloc[ii][0] == 1:
            tp += 1
        else:
            fp += 1

            
print(str((tp+tn) / len(predictedY)) + ' Accuracy')
print(str(fp / (tp + fp)) + ' Precision')

0.6243781094527363 Accuracy
0.3333333333333333 Precision


C:\Users\yshik\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:1059: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
